In [ ]:
import osiris_utils as utils
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import matplotlib as mpl
import pandas as pd
from tqdm import tqdm
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.colors as mcolors
plt.rcParams['font.size'] = 14

In [ ]:
import osiris_utils as ou

In [ ]:
sim=ou.Simulation("/home/exxxx5/Tese/Decks/weibelTestsFinalV2/weibelBG_B0_1/weibelBG_B0_1.in")
sim2=ou.Simulation("/home/exxxx5/Tese/Decks/weibelTestsFinal/weibelBG_B0_1/weibelBG_B0_1.in")

In [ ]:
ou.Diagnostic.plot_3d

In [ ]:
idx=55
fig, ax = sim2["b3"].plot_3d(idx, scale_type="zero_centered", boundaries= [[0, 20], [0, 40], [0, 40]])
ax.xaxis.labelpad = 5
t=idx*0.285*20
ax.set_title('$t={}$$\\  [1 / \\omega_p]$'.format(t))
plt.show()

In [ ]:
B3 = sim["b3"]

fft_B3_x2 = ou.FFT_Diagnostic(B3, [1, 2])
# fft_B3_x2.load_all()


# Assuming B3 is your 3D data and fft_B3_x2 is the FFT result

# Define spatial frequencies for k1 and k2
k1 = np.fft.fftfreq(B3.nx[0], d=B3.dx[0])
k1 = np.fft.fftshift(k1)  # Shift zero frequency to center

k2 = np.fft.fftfreq(B3.nx[1], d=B3.dx[1])
k2 = np.fft.fftshift(k2)  # Shift zero frequency to center

# Power spectrum slice (from the data)
data = fft_B3_x2[idx][:, :, 40]

# Frequency scaling (assuming omega_p/c is the unit for k)
# Scale k1 and k2 to units of omega_p/c
omega_p_over_c = 1  # Adjust as per your context for the plasma frequency
k1_units = k1 * omega_p_over_c
k2_units = k2 * omega_p_over_c

# Plot the power spectrum in k1-k2 space (2D plot)
plt.pcolormesh(k1_units, k2_units, data[:, :].T, cmap='inferno', shading='auto')
plt.xlabel(r'$\lambda_1^{-1} \, [\omega_p / c]$')  # Label for k1
plt.ylabel(r'$\lambda_2^{-1} \, [\omega_p / c]$')  # Label for k2
plt.title(r'Power Spectrum for $x_3 = 20 [c/\omega_p]$')
plt.colorbar(label='Power')
plt.show()

# Plot the power spectrum along k2 (for a specific slice in k1, e.g., k1[40])
plt.plot(k2_units, data[40, :])
plt.xlabel(r'$\lambda_2^{-1} \, [\omega_p / c]$')
plt.ylabel('Power [a. u.]')
# plt.title(r'Power Spectrum for $x_1, x_3 = 20 [c/\omega_p]$')
plt.xlim([-0.3, 0.3])
plt.tight_layout
plt.show()

print(r"Maximum k2: ", k2[np.argmax(data[40, :])], r" $[\omega_p / c]$")


In [ ]:
# Understand velocity of gyrocenter
def plot_hist(track, t=-1, fig=None, ax=None):
    if fig is None or ax is None:
        fig, ax = plt.subplots()
    
    label = ("$t = {:.2f}$".format(track["t"][0, t]) + "$[{}]$".format(track.units["t"]))

    p = np.sqrt(track["p1"][:,t]**2 + track["p2"][:,t]**2 + track["p3"][:,t]**2)
    ax.hist(p, bins=50, label=label)
    ax.set_xlabel("$p$" + "$[{}]$".format(track.units["p3"]))

    plt.legend()

    return fig, ax


fig, ax = plot_hist(sim["test_electrons"]["tracks"], t=-1)
ax.set_title("Gca Correction")
plt.show()

In [ ]:
shot = "000055"


species = "neg_electrons"

In [ ]:
# read data 

folder_path = Path("/home/exxxx5/Tese/Decks/weibelTestsFinalV2/weibelBG_B0_1")

# neg_el = utils.OsirisGridFile(
#     folder_path / ("MS/DENSITY/" + species + "/charge/charge-" + species + "-" + shot + ".h5")
#     )

b1 = utils.OsirisGridFile(
    folder_path / ("MS/FLD/b1/b1-" + shot + ".h5")
    )

b2 = utils.OsirisGridFile(
    folder_path / ("MS/FLD/b2/b2-" + shot + ".h5")
    )

b3 = utils.OsirisGridFile(
    folder_path / ("MS/FLD/b3/b3-" + shot + ".h5")
    )

e1 = utils.OsirisGridFile(
    folder_path / ("MS/FLD/e1/e1-" + shot + ".h5")
    )

e2 = utils.OsirisGridFile(
    folder_path / ("MS/FLD/e2/e2-" + shot + ".h5")
    )

e3 = utils.OsirisGridFile(
    folder_path / ("MS/FLD/e3/e3-" + shot + ".h5")
    )

In [ ]:
# Histograms

def plotHist(path, shot, key, ax):
    species="test_electrons"
    raw_data = utils.OsirisRawFile(
        path / ("MS/RAW/" + species + "/RAW-" + species + "-" + shot + ".h5")
        )
    
    counts, bins = np.histogram(raw_data.data[key], bins=500)
    bin_centers = (bins[:-1] + bins[1:]) / 2

    bin_widths = bins[1:] - bins[:-1]
  
    # Normalize the counts by the bin width to ensure the same area
    normalized_counts = counts / bin_widths

    ax.plot(
        bin_centers, normalized_counts, 
        label=(r"$t={:.2f}$".format(raw_data.time[0]) + r"$\  [{}]$".format(raw_data.time[1]))
        )
    ax.set_xlabel(r"${}$".format(raw_data.axis[key]["long_name"]) + r"$\  [{}]$".format(raw_data.axis[key]["units"]))
    ax.legend()
    return ax

def compare_hists(path1, path2, key):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
    ax1 = plotHist(path1, "000350", key, ax1)
    ax2 = plotHist(path2, "000350", key, ax2)

    ax1 = plotHist(path1, "000380", key, ax1)
    ax2 = plotHist(path2, "000380", key, ax2)

    ax1 = plotHist(path1, "000410", key, ax1)
    ax2 = plotHist(path2, "000410", key, ax2)

    ax1 = plotHist(path1, "000440", key, ax1)
    ax2 = plotHist(path2, "000440", key, ax2)

    ax1 = plotHist(path1, "000470", key, ax1)
    ax2 = plotHist(path2, "000470", key, ax2)

    ax1 = plotHist(path1, "000500", key, ax1)
    ax2 = plotHist(path2, "000500", key, ax2)

    return ax1, ax2
    





In [ ]:
# field and charge maps

def plot(data):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

    # Plot for the first subplot
    im1 = ax1.imshow(data.data[:,:,40].T, aspect="auto", origin="lower", cmap="Reds", extent=[data.grid[0][0], data.grid[0][1], data.grid[1][0], data.grid[1][1]])
    cbar1 = fig.colorbar(im1, ax=ax1)
    cbar1.set_label(r"${}$".format(data.label) + r"$\  [{}]$".format(data.units))
    ax1.set_title(r"$t={:.2f}$".format(data.time[0]) + r"$\  [{}]$".format(data.time[1]))
    ax1.set_xlabel(r"${}$".format(data.axis[0]["long_name"]) + r"$\  [{}]$".format(data.axis[0]["units"]))
    ax1.set_ylabel(r"${}$".format(data.axis[1]["long_name"]) + r"$\  [{}]$".format(data.axis[1]["units"]))

    # Plot for the second subplot
    im2 = ax2.imshow(data.data[40,:,:].T, aspect="auto", origin="lower", cmap="Reds", extent=[data.grid[1][0], data.grid[1][1], data.grid[2][0], data.grid[2][1]])
    cbar2 = fig.colorbar(im2, ax=ax2)
    cbar2.set_label(r"${}$".format(data.label) + r"$\  [{}]$".format(data.units))
    ax2.set_title(r"$t={:.2f}$".format(data.time[0]) + r"$\  [{}]$".format(data.time[1]))
    ax2.set_xlabel(r"${}$".format(data.axis[1]["long_name"]) + r"$\  [{}]$".format(data.axis[1]["units"]))
    ax2.set_ylabel(r"${}$".format(data.axis[2]["long_name"]) + r"$\  [{}]$".format(data.axis[2]["units"]))

    return fig, ax1, ax2



In [ ]:
# Energy plots

# Define column names based on the header
column_names = ['Iter', 'Time', 'B1', 'B2', 'B3', 'E1', 'E2', 'E3']

# Read the data file
fields = pd.read_csv(folder_path / 'HIST/fld_ene'
                   , sep=r'\s+', skiprows=2, names=column_names)

column_names2 = [' Iter', 'Time', 'Total Par.', 'Kin. Energy']


ax = plt.subplot()
sum = fields['Time']*0.
for label, labelLatex in zip(['B1', 'B2', 'B3'], [r'$B_1$', r'$B_2$', r'$B_3$']):
    ax.plot(fields['Time'], fields[label], label=labelLatex)
    sum += fields[label]**2
# ax.plot(fields['Time'], sum, label="$B^2$")

ax.vlines(313.5, ymin=fields['B2'].min(), ymax=fields['B1'].max()*2, colors='k', linestyles='dashed', label='$t={}$'.format(t))

ax.legend()
ax.set_xlabel(r"$t [1 / \omega_p]$")
ax.set_ylabel(r"$0.5\Delta x^3 \Sigma B_i^2 \, [m_e^2 c^5 e^{-2} \omega_p]$")
ax.grid()
plt.yscale('log')
plt.tight_layout()
plt.show()



In [ ]:

# ax = plt.subplot()
# ax.plot(fields['Time'], fields["B1"]**2 + fields["B2"]**2 + fields["B3"]**2, label="cold")
# # ax.plot(fields_Hot['Time'], fields_Hot["B1"]**2 + fields_Hot["B2"]**2 + fields_Hot["B3"]**2, label="hot")

# #ax.plot(fields['Time'][0:200], 1*np.exp(2*0.4*fields['Time'])[0:200], label=r"$1*e^{(2*0.4*t)}$")
# ax.set_title(r"$B^2$")
# plt.yscale('log')
# ax.legend()
# ax.set_xlabel("t")
# ax.set_ylabel("fld_ene")
# ax.grid()
# #ax.set_xlim(350,400)
# #ax.set_ylim(1.25e5,1.27e5)

# plt.show()

# for label in ['B1', 'B2', 'B3', 'E1', 'E2', 'E3']:
#     ax = plt.subplot()
#     ax.plot(fields['Time'], fields[label]**2, label="cold")
#     # ax.plot(fields_Hot['Time'], fields_Hot[label]**2, label="hot")

#     #ax.plot(fields['Time'][0:200], 1*np.exp(2*0.4*fields['Time'])[0:200], label=r"$1*e^{(2*0.4*t)}$")
#     ax.set_title(label+r"$^2$")
#     ax.legend()
#     ax.set_xlabel("t")
#     ax.set_ylabel("fld_ene")
#     ax.grid()
#     plt.yscale('log')
#     plt.show()

In [ ]:
80**3 *8

In [ ]:
# b1_grid = b1.yeeToCellCorner()
# b2_grid = b2.yeeToCellCorner()
# b3_grid = b3.yeeToCellCorner()
# e1_grid = e1.yeeToCellCorner()
# e2_grid = e2.yeeToCellCorner()
# e3_grid = e3.yeeToCellCorner()
#print(np.shape(b1_grid))

sim = utils.Simulation("/home/exxxx5/Tese/Decks/ComparePushers/weibelBG_B0_1/weibelBG_B0_1.in")
centered = utils.FieldCentering_Simulation(sim)

b1_grid=centered["b1"][55]
b2_grid=centered["b2"][55]
b3_grid=centered["b3"][55]
e1_grid=centered["e1"][55]
e2_grid=centered["e2"][55]
e3_grid=centered["e3"][55]


b2 = b1_grid * b1_grid + b2_grid * b2_grid + b3_grid * b3_grid
b = np.sqrt(b2)

e_par = np.abs(e1_grid * b1_grid + e2_grid * b2_grid + e3_grid * b3_grid / b)

e2 = e1_grid * e1_grid + e2_grid * e2_grid + e3_grid * e3_grid
e = np.sqrt(e2)
e_perp =  np.sqrt(e2 - e_par * e_par )


In [ ]:

colors = [
    (0, "black"),   # 0 -> Dark Blue
    (0.4, "blue"), # Mid-range transition
    (0.5, "white"),      # 1 -> White (exactly at 1)
    (0.6, "red"), # Mid-range transition
    (1, "darkred")       # >1 -> Red
]
custom_cmap = LinearSegmentedColormap.from_list("custom_cmap", colors)

def plot_grid_centred(data, label, cmap, vmin=0, vmax = 2):

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

    # Plot for the first subplot
    im1 = ax1.imshow(data[:,:,40].T, aspect="auto", origin="lower", cmap=cmap, 
                     extent=[b1.grid[0][0], b1.grid[0][1], b1.grid[1][0], b1.grid[1][1]], vmin=vmin, vmax=vmax)
    
    cbar1 = fig.colorbar(im1, ax=ax1)
    cbar1.set_label(label)
    ax1.set_title(r"$t={:.2f}$".format(b1.time[0]) + r"$\  [{}]$".format(b1.time[1]))
    ax1.set_xlabel(r"${}$".format(b1.axis[0]["long_name"]) + r"$\  [{}]$".format(b1.axis[0]["units"]))
    ax1.set_ylabel(r"${}$".format(b1.axis[1]["long_name"]) + r"$\  [{}]$".format(b1.axis[1]["units"]))


    # Plot for the second subplot
    im2 = ax2.imshow(data[40,:,:].T, aspect="auto", origin="lower", cmap=cmap, 
                     extent=[b1.grid[1][0], b1.grid[1][1], b1.grid[2][0], b1.grid[2][1]],
                     vmin = vmin, vmax=vmax)
    cbar2 = fig.colorbar(im2, ax=ax2)
    cbar2.set_label(label)
    ax2.set_title(r"$t={:.2f}$".format(b1.time[0]) + r"$\  [{}]$".format(b1.time[1]))
    ax2.set_xlabel(r"${}$".format(b1.axis[1]["long_name"]) + r"$\  [{}]$".format(b1.axis[1]["units"]))
    ax2.set_ylabel(r"${}$".format(b1.axis[2]["long_name"]) + r"$\  [{}]$".format(b1.axis[2]["units"]))

    return fig, ax1, ax2

In [ ]:
fig, ax1, ax2 = plot_grid_centred(e_par/e_perp, r"$\frac{e_{\parallel}}{e_{\perp}}$", "seismic")
plt.tight_layout()
plt.show()

fig, ax1, ax2 = plot_grid_centred(e/b, r"$\frac{e}{b}$", "seismic")
plt.tight_layout()
plt.show()

fig, ax1, ax2 = plot_grid_centred(b, r"$b$", "plasma", vmax=None, vmin=None)
plt.tight_layout()
plt.show()

fig, ax1, ax2 = plot_grid_centred(e, r"$e$", "plasma", vmax=None, vmin=None)
plt.tight_layout()
plt.show()